# 1. Import libraries

In [ ]:
# our scripts
import sys
sys.path.append("./src/semantic")

from src.semantic.perform_clustering import run_all_clustering

# 2. Perform Clustering for all article embeddings

In [ ]:
## Define file paths 
# Path to the category data file
category_file = './data/wikispeedia/wikispeedia_paths-and-graph/categories.tsv'      
# All paths to the embedding data files
MiniLM_L6_v2_file = './data/semantic/output/embeddings/all_MiniLM_L6_v2/embeddings.pkl' 
mpnet_base_v2_file = './data/semantic/output/embeddings/mpnet_base_v2/embeddings.pkl' 
roberta_file = './data/semantic/output/embeddings/roberta/embeddings.pkl' 

## Run clustering on all models and print the results
# all_MiniLM_L6_v2
print('clustering results for MiniLM_L6_v2 results:')
MiniLM_L6_v2_clustering = run_all_clustering(MiniLM_L6_v2_file, category_file)
for model_name, clusters in MiniLM_L6_v2_clustering.items():
    print(f"Model: {model_name}")
    print(len(clusters))

# all_mpnet_base_v2
print('clustering results for mpnet_base_v2 results:')
mpnet_base_v2_clustering = run_all_clustering(mpnet_base_v2_file, category_file)
for model_name, clusters in mpnet_base_v2_clustering.items():
    print(f"Model: {model_name}")
    print(len(clusters))

# roberta
print('clustering results for roberta results:')
roberta_clustering = run_all_clustering(roberta_file, category_file)
for model_name, clusters in roberta_clustering.items():
    print(f"Model: {model_name}")
    print(len(clusters))